In [114]:
import pandas as pd
import numpy as np

In [115]:
data = pd.read_csv('/home/kirill/Desktop/code_blocks_big.csv', sep='\t')
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,['invite people for the kaggle party']
1,"\ndf_train = pd.read_csv('../input/train.csv'),",['bring in the six packs']
2,"\ndf_train.columns,",['check the decoration']
3,"\ndf_train['SalePrice'].describe(),",['descriptive statistics summary']
4,"\nsns.distplot(df_train['SalePrice']);,",['histogram']


Уникальных тегов:

In [116]:
data['tag'].nunique()

102123

Размер:

In [117]:
data['tag'].shape

(213535,)

### Удаление стоп-слов, стемминг, оставляем по одному комментарию

In [103]:
# from nltk import download
# download('stopwords')

In [99]:
from nltk.stem.snowball import SnowballStemmer 
from nltk.corpus import stopwords

In [104]:
st_wr = set(stopwords.words('english'))

In [95]:
stemmer = SnowballStemmer('english')

In [113]:
stemmer.stem('kirill eats grass on the floors, long word endings')

'kirill eats grass on the floors, long word end'

**Важно:** если у вас в табличке есть колонка 'Unnamed 0:' или вообще какая то лишняя, то дропайте ее до этого цикла (почему-то это сильно влияет на производительность)

Тут из нескольких комментариев выбирается самый короткий (обосновано тем, что тогда он с большей вероятностью встречается часто ака больше похож на метку). Отсекаются пробелы и всякие пустые метки.

In [133]:
import re
def no_punc(tag):
    return re.sub('[^A-Za-z0-9]+_', '',tag)

In [118]:
from tqdm import tqdm
all_to_del = set(['', "['']", ' ', '  ', "[' ']", '        ', '     ', '    ', '   ', "['', ' ']", '       ', '      ', \
             '             ', '         ', '          ', "']",\
                 "['   ", "['", "[' ", "['  ", "['    ", "['    "])
rows_to_del = []

In [134]:
for i in tqdm(range(data.shape[0])):
    my_string = data.loc[i, 'tag']
    
    if (my_string is not np.nan) and my_string is not None:
        
        min_len = len(data.loc[i, 'tag']) + 10
        min_el = None
        s = set(list(map(str, my_string.split("', '"))))
        
        for el in s:
            
            # getting rid of punctuation
            el = no_punc(el)
            
            if el not in all_to_del:
                
                # getting rid of stop words
                el = [w.lower() for w in el.split() if w.lower() not in st_wr]
                el = ' '.join(el)
                
                # stemming 
                el = stemmer.stem(el)
                
                if len(el) < min_len:
                    min_len = len(el)
                    min_el = el

        if min_el is not None:
            data.loc[i, 'tag'] = min_el
        else:
            rows_to_del.append(i)
    else:
        rows_to_del.append(i)

100%|██████████| 213535/213535 [01:51<00:00, 1907.60it/s]


Было:

In [135]:
data.shape

(213535, 2)

In [136]:
data = data.drop(rows_to_del)

Осталось:

In [137]:
data.shape

(212459, 2)

In [138]:
data = data.reset_index()

In [139]:
data.drop(columns=['index'], axis=1, inplace=True)
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,['invite people kaggle party']
1,"\ndf_train = pd.read_csv('../input/train.csv'),",['bring six packs']
2,"\ndf_train.columns,",['check decoration']
3,"\ndf_train['SalePrice'].describe(),",['descriptive statistics summary']
4,"\nsns.distplot(df_train['SalePrice']);,",['histogram']


________

### Предобработка текста

In [140]:
import re
def clear_tag(tag):
    tag = tag.replace(' ', '_')
    return re.sub('[^A-Za-z0-9_]+', '',tag) # replace spaces with "_" and removes other punctuation

In [141]:
data['tag'] = list(map(clear_tag, data['tag'])) # apply function
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,invite_people_kaggle_party
1,"\ndf_train = pd.read_csv('../input/train.csv'),",bring_six_packs
2,"\ndf_train.columns,",check_decoration
3,"\ndf_train['SalePrice'].describe(),",descriptive_statistics_summary
4,"\nsns.distplot(df_train['SalePrice']);,",histogram


In [142]:
def clear_first(tag):
    if tag is not np.nan and tag is not None \
            and len(tag) > 0 and tag[0] == '_':
        return tag[1:]
    else:
        return tag

In [143]:
data['tag'] = list(map(clear_first, data['tag'])) # clears first symbol if it's "_"
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,invite_people_kaggle_party
1,"\ndf_train = pd.read_csv('../input/train.csv'),",bring_six_packs
2,"\ndf_train.columns,",check_decoration
3,"\ndf_train['SalePrice'].describe(),",descriptive_statistics_summary
4,"\nsns.distplot(df_train['SalePrice']);,",histogram


In [144]:
data = data[data['tag'] != '']
data = data[data['tag'] != '_']
data = data[data['tag'] != '__']
data = data[data['tag'] != '___']
data = data[data['tag'] != '____']
data = data[data['tag'] != '_____']

Осталось уже:

In [145]:
data.shape

(210798, 2)

_____________

### Режем по длинне метки:

Здесь проверка, чтобы не было копипаста, потому что комментарий длиннее 30 символов совпавший так много раз явно коппаст:

In [146]:
data = data[data['tag'].str.len() < 30]

In [147]:
print(data.shape)
print(data['tag'].nunique())

(125317, 2)
39130


### Оставим какие-то самые популярные метки

Из самых популярных 5к меток выкину варианты длиннее n

In [148]:
data['tag'].nunique()

39130

In [149]:
from collections import Counter

mc = Counter(data['tag']).most_common(5000)

In [150]:
strange_to_del = []

Это работает не очень 

Если уникальных блоков кода с этим комментарием меньше 75 (естественно стоит эту константу пробовать менять), то удалим эту метку тк это скорее всего копипаст:

Так делаем среди самых популярных 5к тегов чтобы не тратить время на совсем не популярные метки

In [151]:
for el in tqdm(mc):
    if data[data['tag']==el[0]]['code_block'].nunique() < 75:
        strange_to_del.append(el[0])

100%|██████████| 5000/5000 [00:26<00:00, 186.22it/s]


In [152]:
for el in tqdm(strange_to_del):
    data = data[data['tag'] != el]

100%|██████████| 4933/4933 [01:07<00:00, 73.39it/s]


Здесь естественно теряется много данных:

In [153]:
print(data.shape)
print(data['tag'].nunique())

(66264, 2)
34197


Заметим, что выкинули 4933 меток из самых популярных 5000, оставим 30 самых популярных:

Ниже

In [156]:
Counter(data['tag']).most_common(30)

[('linear_algebra', 3017),
 ('plot', 2962),
 ('model', 919),
 ('check_answer', 582),
 ('create_x', 553),
 ('code', 537),
 ('predict', 527),
 ('read_data', 518),
 ('load_data', 431),
 ('train', 421),
 ('uncomment_them', 397),
 ('random_forest', 358),
 ('visual', 331),
 ('feature_engin', 326),
 ('submiss', 315),
 ('preprocess', 310),
 ('eda', 268),
 ('test', 267),
 ('data_preprocess', 232),
 ('check_answers', 227),
 ('data', 220),
 ('train_model', 216),
 ('xgboost', 205),
 ('import', 205),
 ('missing_valu', 189),
 ('input', 183),
 ('model_build', 177),
 ('fit_model', 170),
 ('data_visu', 161),
 ('blend', 158)]

In [157]:
# all_to_del - задан вначале ноутбука

In [158]:
for to_del in all_to_del:
    data = data[data['tag'] != to_del]

In [159]:
print(data.shape)
print(data['tag'].nunique())

(66264, 2)
34197


In [161]:
num_tags_to_drop = data['tag'].nunique() - 30
num_tags_to_drop

34167

Выкидываю самые непопулярные (nunique - 30) меток:

In [162]:
least_c_tags = set(Counter(data['tag']).most_common()[-num_tags_to_drop:])

In [163]:
for el in tqdm(least_c_tags):
    data = data[data['tag'] != el[0]]

100%|██████████| 34167/34167 [04:56<00:00, 115.38it/s]


Смотрим что осталось:

In [165]:
print(data.shape)
print(data['tag'].nunique())

(15382, 2)
30


In [166]:
ready_data = \
data.reset_index().drop(columns=['index'], axis=1)
#ready_data = ready_data.drop(columns=['level_0'], axis=1)

In [167]:
Counter(ready_data['tag']).most_common()[:10]

[('linear_algebra', 3017),
 ('plot', 2962),
 ('model', 919),
 ('check_answer', 582),
 ('create_x', 553),
 ('code', 537),
 ('predict', 527),
 ('read_data', 518),
 ('load_data', 431),
 ('train', 421)]

In [168]:
Counter(ready_data['tag']).most_common()[-10:]

[('data', 220),
 ('train_model', 216),
 ('xgboost', 205),
 ('import', 205),
 ('missing_valu', 189),
 ('input', 183),
 ('model_build', 177),
 ('fit_model', 170),
 ('data_visu', 161),
 ('blend', 158)]

На данном этапе все работает очень плохо, поэтому давайте попробуем руками с метками поработать...

In [169]:
ready_data['tag'].unique()

array(['visual', 'random_forest', 'linear_algebra', 'read_data',
       'train_model', 'plot', 'predict', 'xgboost', 'preprocess',
       'load_data', 'import', 'feature_engin', 'model', 'fit_model',
       'data', 'submiss', 'test', 'missing_valu', 'train', 'input',
       'data_visu', 'blend', 'eda', 'check_answer', 'data_preprocess',
       'code', 'model_build', 'create_x', 'check_answers',
       'uncomment_them'], dtype=object)

Вариант для стемминга:

In [171]:
# train, train_model, model_build, model, fit_model
# train_model, fit_model -> train
# 'data_visu' -> visual
# model_building -> model_build

In [170]:
ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'train_model', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'fit_model', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'data_preprocess', 'tag'] = 'preprocess'
ready_data.loc[ready_data['tag'] == 'check_answers', 'tag'] = 'check_answer'
ready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'
ready_data.loc[ready_data['tag'] == 'data_visu', 'tag'] = 'visual'
ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model_build'

jut_inds = ready_data[ready_data['tag'] == 'uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

Без стемминга:

In [52]:
# data -> load_data
# exploratory_data_analysis, data_exploration -> eda
# data_preprocessing -> preprocessing
# training -> train
# check_your_answer, check_your_answers -> check_answer
# model -> model_building
# just_uncomment_them -> trash


##################### maybe
# prediction - test ?

In [172]:
'''ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'
ready_data.loc[ready_data['tag'] == 'data_exploration', 'tag'] = 'eda'
ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
ready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'check_your_answer', 'tag'] = 'check_answer'
ready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_answer'
ready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'

jut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)'''

"ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'\nready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'\nready_data.loc[ready_data['tag'] == 'data_exploration', 'tag'] = 'eda'\nready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'\nready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'\nready_data.loc[ready_data['tag'] == 'check_your_answer', 'tag'] = 'check_answer'\nready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_answer'\nready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'\n\njut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index\nready_data.drop(index=jut_inds, axis=0, inplace=True)"

In [190]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(14982, 2)
23


[('linear_algebra', 3017),
 ('plot', 2959),
 ('model_building', 919),
 ('check_answer', 809),
 ('train', 807),
 ('load_data', 651),
 ('create_x', 553),
 ('preprocess', 542),
 ('code', 537),
 ('predict', 527),
 ('read_data', 518),
 ('visual', 492),
 ('random_forest', 358),
 ('feature_engin', 326),
 ('submiss', 315),
 ('eda', 268),
 ('test', 267),
 ('xgboost', 205),
 ('import', 205),
 ('missing_valu', 189),
 ('input', 183),
 ('model_build', 177),
 ('blend', 158)]

Слишком много линейной алгебры и др, давайте из них выберем по 800 объектов (не костыль, а эксперимент)

Удаляем unbalance:

In [192]:
todel_mc1 = np.random.choice(
    np.array(ready_data[ready_data['tag'] == 'linear_algebra'].index),
    3017 - 800,
    replace = False
)
todel_mc2 = np.random.choice(
    np.array(ready_data[ready_data['tag'] == 'plot'].index),
    2959 - 800,
    replace = False
)

ready_data.drop(index=todel_mc1, inplace=True, axis=0)
ready_data.drop(index=todel_mc2, inplace=True, axis=0)

In [196]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(10606, 2)
22


[('model_build', 1096),
 ('check_answer', 809),
 ('train', 807),
 ('plot', 800),
 ('linear_algebra', 800),
 ('load_data', 651),
 ('create_x', 553),
 ('preprocess', 542),
 ('code', 537),
 ('predict', 527),
 ('read_data', 518),
 ('visual', 492),
 ('random_forest', 358),
 ('feature_engin', 326),
 ('submiss', 315),
 ('eda', 268),
 ('test', 267),
 ('xgboost', 205),
 ('import', 205),
 ('missing_valu', 189),
 ('input', 183),
 ('blend', 158)]

____________

Соотношения train, test, val = 8.5 / 1.5 / 1.5

Поделим нацело:

In [199]:
(ready_data.shape[0] - 2) / 11

964.0

In [200]:
ready_data = ready_data.drop(ready_data.tail(2).index)

In [201]:
ready_data.shape[0] / 11

964.0

То есть размер трейна:

In [203]:
964.0 * 8

7712.0

Стратифайдом делю на трейн, тест, сплит

In [204]:
# https://stackoverflow.com/questions/40829137/stratified-train-validation-test-split-in-scikit-learn

from sklearn.model_selection import StratifiedShuffleSplit
SEED = 42001

split = StratifiedShuffleSplit(n_splits=1, test_size=3/11, random_state=SEED)
for train_index, test_valid_index in split.split(\
                                                 ready_data.drop('tag', axis=1), \
                                                 ready_data['tag']\
                                                ):
    train_set = ready_data.iloc[train_index]
    test_valid_set = ready_data.iloc[test_valid_index]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
for test_index, valid_index in split2.split(test_valid_set, test_valid_set['tag']):
    test_set = test_valid_set.iloc[test_index]
    valid_set = test_valid_set.iloc[valid_index]

In [205]:
train_set = train_set.reset_index()
test_set = test_set.reset_index()
valid_set = valid_set.reset_index()

______

Далее все '\n' заменяются на '\n\t', а также вначале каждого сниппета приписывается

'def tag():\n\t'

Каждый сниппет записывается в **отдельный файл** с названием номер_итерации.py в каталог трей, тест или валидейшн.

In [208]:
def add_tabs(exmpl):
    new_exmpl = str()
    for i in range(len(exmpl)):
        if (exmpl[i]=='\n'):
            new_exmpl += '\n\t'
        else:
            new_exmpl += exmpl[i]
    return new_exmpl

train:

In [209]:
for i in range(train_set.shape[0]):
        exmpl = train_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(train_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_train/func'+str(i)+'.py', 'w'))

test:

In [210]:
for i in range(test_set.shape[0]):
        exmpl = test_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(test_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_test/func'+str(i)+'.py', 'w'))

valid:

In [211]:
for i in range(valid_set.shape[0]):
        exmpl = valid_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(valid_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_val/func'+str(i)+'.py', 'w'))

In [212]:
len(valid_set)

1446

_______

Предобработка для просто 30 строк кода посложнее, пока ее нет.